## Scraping de texto sobre a lei rouanet

###  Daniel Silva Agostinho - 470435

#### Retirando dados do twitter utilizando a biblioteca Selenium

In [3]:
#Script

import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from msedge.selenium_tools import Edge, EdgeOptions
import datetime

def transform_number(st_number):
    split_numb = st_number.split(" ")
    try:
        number = float(split_numb[0].replace(',', '.'))
    except ValueError:
        return 0
    if(len(split_numb) == 2):
        multiplicator = split_numb[1]

        if(multiplicator == "mil"):
            return int(number * 1000)
        if(multiplicator == "mi"):
            return int(number * 1000000)
    return int(number)

def get_data(tweet):
    #username
    try:
        username = tweet.find_element_by_xpath('./div/div/div/div[2]/div[2]/div[1]//span').text
    except NoSuchElementException:
        return
    except StaleElementReferenceException:
        return
        
    #arroba, ex: @daniel_agostin
    try:
        arroba = tweet.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    except StaleElementReferenceException:
        return
    
    try:
        #dia
        dia = tweet.find_element_by_xpath('./div/div/div/div[2]/div[2]/div[1]//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    except StaleElementReferenceException:
        return
    
    #conteudo
    try:
        conteudo = tweet.find_element_by_xpath('./div/div/div/div[2]/div[2]/div[2]//div[@lang="pt"]').text
    except NoSuchElementException:
        return
    except StaleElementReferenceException:
        return

    #quantidade de comentarios
    try:
        comentarios = tweet.find_element_by_xpath('./div/div/div/div[2]/div[2]/div[2]/div[3]/div/div[1]//span').text
        comentarios = transform_number(comentarios)
    except NoSuchElementException:
        comentarios = 0
    except StaleElementReferenceException:
        return
        
    #quantidade de retweets
    try:
        ret = tweet.find_element_by_xpath('./div/div/div/div[2]/div[2]/div[2]/div[3]/div/div[2]//span').text
        ret = transform_number(ret)
    except NoSuchElementException:
        ret = 0
    except StaleElementReferenceException:
        return
    
    #quantidade de likes
    try:
        likes = tweet.find_element_by_xpath('./div/div/div/div[2]/div[2]/div[2]/div[3]/div/div[3]//span').text
        likes = transform_number(likes)
    except NoSuchElementException:
        likes = 0
    except StaleElementReferenceException:   
        return
        
    tweet_id = arroba + dia
    
    tweet_tuple = {'id':tweet_id,"username":username, "arroba":arroba, "date_time":dia, "texto":conteudo, "coment_cont":comentarios, "ret_cont":ret, "likes_cont":likes}
    
    return tweet_tuple

options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)

#acesssando login page do twitter
driver.get("https://twitter.com/i/flow/login")

#parada pra logar no twitter
a = input()

barra_pesquisa = driver.find_element_by_xpath('//input[@placeholder="Buscar no Twitter"]')
barra_pesquisa.send_keys('lei rouanet until:2021-11-30 since:2020-07-30')
barra_pesquisa.send_keys(Keys.RETURN)

sleep(1)

#Aba de tweets mais recentes
mais_recentes = driver.find_element_by_xpath('//div[@role="tablist"]/div[2]/a')
mais_recentes.click()

#vetor de tweets
tweet_data = []

#vetor de ids, evitar de pegar o mesmo tweet mais de uma vez
ids = set()

#quantidade minima de tweets a ser minerada
quantidade_min = 10

while (len(tweet_data) < quantidade_min):
    
    if len(tweet_data)%500 < 10:
        print("len atual: ", len(tweet_data), "as: ",datetime.datetime.now().time())
    
    #carrego os tweets viziveis
    tweets = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
    
    for tweet in tweets[-10:]:
        tweet_tuple = get_data(tweet)
        if tweet_tuple:
            if tweet_tuple['id'] not in ids:
                tweet_data.append(tweet_tuple)
                ids.add(tweet_tuple['id'])
            
    
    driver.execute_script('window.scrollBy(0, 7000);')
    sleep(1)
    
#Escrevendo csv com os dados
with open('twitter_video_1.csv', 'w', newline='', encoding = 'utf-8') as f:
    writer = csv.writer(f)
    for tweetD in tweet_data:
        writer.writerow(list(tweetD.values()))  

len atual:  0 as:  19:47:13.284713
len atual:  0 as:  19:47:14.316951
len atual:  0 as:  19:47:15.335179
len atual:  7 as:  19:47:17.149588


In [4]:
len(tweet_data)

4001